In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf
import pandas as pd
import os
#import pickle
from utility import *
from sklearn.metrics import classification_report
#from sklearn.model_selection import GridSearchCV

In [2]:
tf.logging.set_verbosity(tf.logging.INFO)


def mlp_model_fn(features, labels, mode, params):
  """Model function for MLP."""
  
  #TODO: need works on this
  config = params


  # Input Layer
  
  input_layer = tf.reshape( features["x"], [-1, features["x"].shape[1] ] )
  #print ('feature x', features["x"])
  #print ('feature x shape', features["x"].shape)
  #print ('reshape:', input_layer)
  #print ('reshape shape:', input_layer.shape)
  #trans = tf.string_to_number(input_layer)
  #print ('trans reshape:', trans)
  #print ('reshape shape:', input_layer.shape)



  # Dense Layers

  hidden1 = tf.layers.dense(inputs=features["x"], units=config['n_hidden1'], activation=tf.nn.relu)
  drop_h1 = tf.layers.dropout(
      inputs=hidden1, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)
  hidden2 = tf.layers.dense(inputs=drop_h1, units=config['n_hidden2'], activation=tf.nn.relu)
  drop_h2 = tf.layers.dropout(
      inputs=hidden2, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)
  if config['n_hidden3'] != None:
    hidden3 = tf.layers.dense(inputs=drop_h2, units=config['n_hidden3'], activation=tf.nn.relu)
    drop_h3 = tf.layers.dropout(
        inputs=hidden3, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=drop_h3, units=config['nclasses'])
  else:
    logits = tf.layers.dense(inputs=drop_h2, units=config['nclasses'])
    
  predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }


  

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=config['nclasses'])
  print (onehot_labels)
  print (logits)
  loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)


  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



In [3]:
unused_argv = ['pilot_mlp_newAPI.py', 'temp_data.csv', 'model/', 1024, 512, 256]

filename = unused_argv[1]


if os.path.isfile('ft_to_idx.npy') and os.path.isfile('nclasses.npy') and os.path.isfile('group_data.npy'):
    ft_to_idx = np.load('ft_to_idx.npy')
    ft_to_idx = ft_to_idx.item()
    nclasses = np.load('nclasses.npy')
    #f = open('group_data.pkl','r')
    #group_data = pickle.load('group_data.pkl')
    #group_data = np.load('group_data.npy') 
    group_data = np.load('toy_data.npy') # for proof of algo purpose, real use case should use the above line
    group_data = group_data.item()

else:
    ft_to_idx, nclasses, group_data = prepare_file(filename)
    np.save("ft_to_idx", ft_to_idx)
    np.save("nclasses", nclasses)
    #f = open('group_data.pkl','w')
    #pickle.dump(group_data, f)
    np.save("group_data", group_data)

#gen train sets
train_full, test = train_dev_split(group_data, proportion = 0.8, thre=1) # should be 1000 or so
train, dev = train_dev_split(train_full, proportion = 0.8, thre=1) # should be 1000 or so
train_full_data, train_full_labels = gen_feed(train_full, ft_to_idx, upper_limit=5000)
train_data, train_labels = gen_feed(train, ft_to_idx, upper_limit=5000)
dev_data, dev_labels = gen_feed(dev, ft_to_idx, upper_limit=5000)
test_data, test_labels = gen_feed(test, ft_to_idx, upper_limit=5000)
train_full_data = train_full_data.astype(np.float32)
train_data = train_data.astype(np.float32) 
dev_data = dev_data.astype(np.float32)
test_data = test_data.astype(np.float32) 
'''
train_data, train_labels = np.zeros((1, group_data['application/pdf'].shape[1]-1)), np.zeros((1,))
for i in range(100):
tmp_data, tmp_labels = subsampled_batch(ft_to_idx, group_data, class_size=100)
train_data = np.vstack((train_data, tmp_data))
train_labels = np.hstack((train_labels, tmp_labels))
train_data = np.delete(train_data,0,0)
train_labels = np.delete(train_labels,0,0)
train_data = train_data.astype(np.float32)
'''
#gen dev set

'''eval_data, eval_labels = subsampled_batch(ft_to_idx, group_data, class_size=100)
eval_data = eval_data.astype(np.float32) 
'''
# set config
config = {}
config['nclasses'] = int(nclasses)
config['model_dir'] = unused_argv[2]
config['n_hidden1'] = int(unused_argv[3])
config['n_hidden2'] = int(unused_argv[4])
config['n_hidden3'] = int(unused_argv[5])

In [5]:
from itertools import product
hidden_units = [[1024,1024]]#,[1024,512,256],[2048,2048],[2048,1024,512]]
batch_sizes = [16,32]#,64,256,512]
dropouts = [0.4,0.2]

#reset config, defined in previous block
config = {}
config['nclasses'] = int(nclasses)
results = []
for hidden, batch_size, dropout in product(hidden_units,batch_sizes,dropouts):
    config['n_hidden1'] = hidden[0]
    config['n_hidden2'] = hidden[1]
    try: config['n_hidden3'] = hidden[2]
    except: config['n_hidden3'] = None
    config['dropout'] = dropout
    # Create the Estimator
    mlp_classifier = tf.estimator.Estimator(
    model_fn=mlp_model_fn, model_dir=None, params=config)

    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=1000)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=batch_size,
      num_epochs=None,
      shuffle=True)

    mlp_classifier.train(
      input_fn=train_input_fn,
      steps=200,
      hooks=[logging_hook])
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": dev_data},
        y=dev_labels,
        num_epochs=1,
        shuffle=False)

    eval_results = mlp_classifier.evaluate(input_fn=eval_input_fn)
    print ('----setting----')
    print ('hidden units:',hidden)
    print ('batch_size:', batch_size)
    print ('dropout:', dropout)
    print ('----performance----')
    print(eval_results)
    results.append((eval_results['accuracy'], hidden, batch_size, dropout))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11ac75990>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpN9FVV3', '_save_summary_steps': 100}
Tensor("one_hot:0", shape=(16, 122), dtype=float32)
Tensor("dense_3/BiasAdd:0", shape=(16, 122), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpN9FVV3/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.00843799  0.00826439  0.00682772 ...,  0.00835717  0.00936133
   0.00759

In [6]:
results = sorted(results, key=lambda x: x[0], reverse=True)
print (results)

[(0.023809524, [1024, 1024], 16, 0.2), (0.017857144, [1024, 1024], 32, 0.2), (0.011904762, [1024, 1024], 16, 0.4), (0.011904762, [1024, 1024], 32, 0.4)]


In [8]:
best_param = results[0]
hidden = best_param[1]
config['n_hidden1'] = hidden[0]
config['n_hidden2'] = hidden[1]
try: config['n_hidden3'] = hidden[2]
except: config['n_hidden3'] = None
batch_size = best_param[2]
config['dropout'] = best_param[3]

# Create the Estimator
cnn_classifier = tf.estimator.Estimator(
model_fn=mlp_model_fn, model_dir=None, params=config)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=2000) 

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": train_full_data},
  y=train_full_labels,
  batch_size=batch_size,
  num_epochs=None,
  shuffle=True)

mlp_classifier.train(
  input_fn=train_input_fn,
  steps=200, # 60000
  hooks=[logging_hook])

pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    num_epochs=1,
    shuffle=False)
predictions = mlp_classifier.predict(input_fn=pred_input_fn)
predictions = list(p["classes"] for p in predictions)

print (classification_report(test_labels, predictions))
print 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x119b9b110>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpTN9N0g', '_save_summary_steps': 100}
Tensor("one_hot:0", shape=(16, 122), dtype=float32)
Tensor("dense_3/BiasAdd:0", shape=(16, 122), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpMIQl5f/model.ckpt-200
INFO:tensorflow:Saving checkpoints for 201 into /var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpMIQl5f/mod

//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
